In [ ]:
import requests, json, io, urllib
from zipfile import ZipFile
#import zipfile
#from contextlib import closing
import urllib.request as ur
import pandas as pd
import subprocess
from simpledbf import Dbf5
from bis2 import gc2
import string

q='''https://www.sciencebase.gov/catalog/items?filter=tags={%22scheme%22:%22BIS%22,%22name%22:%22NHDPlusV2.1%22}&fields=files,id,tags&format=json'''
#Returns 
nhdItems = requests.get(q).json()

# Set up the actions/targets for this particular instance
thisRun = {}
thisRun["instance"] = "DataDistillery"
thisRun["db"] = "BCB"
thisRun["baseURL"] = gc2.sqlAPI(thisRun["instance"],thisRun["db"])
thisRun["schema"] = "sfr"
thisRun["commitToDB"] = False


#Query comids already in the table
q_recordToSearch = "SELECT comid as lookup FROM nhd.nhdplusv2_plusflowlinevaa " 
recordToSearch = requests.get(thisRun["baseURL"]+"&q="+q_recordToSearch).json()
p = recordToSearch['features']
lookup = []
#for record returned, add record to list
for f in p:
    lookup.append(f['properties']['lookup'])
lenList = len(lookup)
cntExisting = 0 
existingList = []
    


for item in nhdItems['items']:
    #Identify which NHD Region via tags 
    for tag in item['tags']:
        if 'Reg' in tag['name']:
            region = tag['name']
    #Look at files and find NHDPlusAttributes File
    for file in item['files']:
        fileName = file['name']
        if 'NHDPlusAttributes' in fileName:
            #Download file
            print ('Retrieving region ' + region + ', file:' + fileName)
            fileUrl = file['url']
            ur.urlretrieve(fileUrl, fileName)
            #Unzip file.  Ideally this will be transformed to work in memory
            subprocess.call(r'"C:\Program Files\7-Zip\7z.exe" x ' + fileName)
            
            #Convert dbf to dataframe
            dbf = Dbf5('PlusFlowLineVAA.dbf')
            df = dbf.to_dataframe()
            for row in df.itertuples():
                if row.ComID in lookup:
                    continue
                else:
                    q = "insert into nhd.nhdplusv2_plusflowlinevaa \
                    (comid, fdate, streamleve, streamorde, streamcalc, fromnode, tonode, hydroseq, levelpath1, \
                    pathlength, terminalpat, arbolatesu, divergence, startflag, terminalfl, dnlevel,  thinnercod, \
                    uplevelpat, uphydroseq, dnlevelpat, dnminorhyd, dndraincou, dnhydroseq, frommeas, tomeas, \
                    reachcode, lengthkm, fcode, rtndiv, outdiv, diveeffect, vpuin, vpuout, areasqkm, totdasqkm, \
                    divdasqkm, tidal, totma, wbareatype) VALUES \
                    ('" + str(row.ComID) + "' ,'" + str(row.Fdate) + "' ,'" + str(row.StreamLeve) + "' ,'"  \
                    + str(row.StreamOrde) + "' ,'" + str(row.StreamCalc) + "' ,'" + str(row.FromNode) + "' ,'" \
                    + str(row.ToNode) + "' ,'" + str(row.Hydroseq) + "' ,'" + str(row.LevelPathI) +  "' ,'" \
                    + str(row.Pathlength)+ "' ,'" + str(row.TerminalPa) + "' ,'" + str(row.ArbolateSu) + "' ,'" \
                    + str(row.Divergence) + "' ,'" + str(row.StartFlag) + "' ,'" + str(row.TerminalFl) + "' ,'" \
                    + str(row.DnLevel) + "' ,'" +   str(row.ThinnerCod) + "' ,'" +   str(row.UpLevelPat) + "' ,'" \
                    + str(row.UpHydroseq) + "' ,'" + str(row.DnLevelPat) + "' ,'" + str(row.DnMinorHyd) + "' ,'" \
                    + str(row.DnDrainCou) + "' ,'" + str(row.DnHydroseq) + "' ,'" + str(row.FromMeas) + "' ,'" \
                    + str(row.ToMeas) + "' ,'" + str(row.ReachCode) + "' ,'" + str(row.LengthKM) + "' ,'" \
                    + str(row.Fcode) + "' ,'" + str(row.RtnDiv) + "' ,'" + str(row.OutDiv) + "' ,'" \
                    + str(row.DivEffect) + "' ,'" + str(row.VPUIn) + "' ,'" + str(row.VPUOut) + "' ,'" \
                    + str(row.AreaSqKM) + "' ,'" + str(row.TotDASqKM) + "' ,'" + str(row.DivDASqKM) + "' ,'" \
                    + str(row.Tidal) + "' ,'" + str(row.TOTMA) + "' ,'" + str(row.WBAreaType) + "')"
                    gc2Key = gc2.gc2Keys["datadistillery_bcb"]
                    payload = "q=%s&key=%s"%(q,gc2Key)
                    url= gc2.baseURLs["sqlapi_datadistillery_bcb"]
                    finalUrl = url + '?q=' +q + '&key=' + gc2Key
                                
                    try:
                        #r = requests.post(url,data=payload)
                        r = requests.get(finalUrl)
                        #r.get(finalUrl).json()
                        status = str(r.status_code)
                        if r.status_code == 200:
                            continue
                        else:
                            print (str(row.ComID) + ' failed with code: ' + status)
                    except requests.exceptions.HTTPError:
                        print (str(row.ComID) + ' failed with code: ' + status)
   

In [1]:
import requests, json, io, urllib
from zipfile import ZipFile
import zipfile
#from contextlib import closing
import urllib.request as ur
import pandas as pd
import subprocess
from simpledbf import Dbf5
from bis2 import gc2
import string
import tempfile
import os

q='''https://www.sciencebase.gov/catalog/items?filter=tags={%22scheme%22:%22BIS%22,%22name%22:%22NHDPlusV2.1%22}&fields=files,id,tags&format=json'''
#Returns 
nhdItems = requests.get(q).json()

# Set up the actions/targets for this particular instance
thisRun = {}
thisRun["instance"] = "DataDistillery"
thisRun["db"] = "BCB"
thisRun["baseURL"] = gc2.sqlAPI(thisRun["instance"],thisRun["db"])
thisRun["schema"] = "sfr"
thisRun["commitToDB"] = False


dfAll = None

for item in nhdItems['items']:
    #Identify which NHD Region via tags 
    for tag in item['tags']:
        if 'Reg' in tag['name']:
            region = tag['name']
            regFolder = '/' + region
            if not os.path.exists(regFolder):
                os.makedirs(regFolder)
                os.chdir(regFolder)

    #Look at files and find NHDPlusAttributes File
    for file in item['files']:
        fileName = file['name']
        if 'NHDPlusAttributes' in fileName:
            print ('Retrieving region ' + region + ', file:' + fileName)
            fileUrl = file['url']
            ur.urlretrieve(fileUrl, fileName)
            subprocess.call(r'"C:\Program Files\7-Zip\7z.exe" x ' + fileName)
                #with zipfile.ZipFile(fileName, "r") as zip_ref:
                #    zip_ref.extractall(tmpdirname)
                #    zip_ref.close()
                #Convert dbf to dataframe
            dbf = Dbf5('PlusFlowLineVAA.dbf')
            df = dbf.to_dataframe()
                            
            if dfAll is None:
                dfAll = df
            else:
                dfAll.append(df)
                    
   

Retrieving region Reg14, file:NHDPlusAttributes.zip
Retrieving region Reg8, file:NHDPlusAttributes.zip
Retrieving region Reg13, file:NHDPlusAttributes.zip
Retrieving region Reg2, file:NHDPlusAttributes.zip
Retrieving region Reg18, file:NHDPlusAttributes.zip
Retrieving region Reg17, file:NHDPlusAttributes.zip
Retrieving region Reg7, file:NHDPlusAttributes.zip
Retrieving region Reg4, file:NHDPlusAttributes.zip
Retrieving region Reg9, file:NHDPlusAttributes.zip
Retrieving region Reg1, file:NHDPlusAttributes.zip
Retrieving region Reg12, file:NHDPlusAttributes.zip
Retrieving region Reg3, file:NHDPlusAttributes03N.zip
Retrieving region Reg3, file:NHDPlusAttributes03S.zip
Retrieving region Reg3, file:NHDPlusAttributes03W.zip
Retrieving region Reg5, file:NHDPlusAttributes.zip
Retrieving region Reg15, file:NHDPlusAttributes.zip
Retrieving region Reg10L, file:NHDPlusAttributes.zip
Retrieving region Reg16, file:NHDPlusAttributes.zip
Retrieving region Reg10U, file:NHDPlusAttributes.zip
Retrieving 

In [9]:
import requests, json, io, urllib
from zipfile import ZipFile
import zipfile
#from contextlib import closing
import urllib.request as ur
import pandas as pd
import subprocess
from simpledbf import Dbf5
from bis2 import gc2
import string
import tempfile
import os

q='''https://www.sciencebase.gov/catalog/items?filter=tags={%22scheme%22:%22BIS%22,%22name%22:%22NHDPlusV2.1%22}&fields=files,id,tags&format=json'''
#Returns 
nhdItems = requests.get(q).json()

# Set up the actions/targets for this particular instance
thisRun = {}
thisRun["instance"] = "DataDistillery"
thisRun["db"] = "BCB"
thisRun["baseURL"] = gc2.sqlAPI(thisRun["instance"],thisRun["db"])
thisRun["schema"] = "sfr"
thisRun["commitToDB"] = False


dfAll = None

for item in nhdItems['items']:
    #Identify which NHD Region via tags 
    for tag in item['tags']:
        if 'Reg' in tag['name']:
            region = tag['name']
            if not os.path.exists('proc'):
                os.makedirs('proc')
                os.chdir('proc')

    #Look at files and find NHDPlusAttributes File
    for file in item['files']:
        fileName = file['name']
        if 'NHDPlusAttributes' in fileName:
            print ('Retrieving region ' + region + ', file:' + fileName)
            fileUrl = file['url']
            ur.urlretrieve(fileUrl, fileName)
            subprocess.call(r'"C:\Program Files\7-Zip\7z.exe" x ' + fileName)
                #with zipfile.ZipFile(fileName, "r") as zip_ref:
                #    zip_ref.extractall(tmpdirname)
                #    zip_ref.close()
                #Convert dbf to dataframe
            dbf = Dbf5('PlusFlowLineVAA.dbf')
            dbf = dbf.to_dataframe()
                                       
            if dfAll is None:
                dfAll = df
            else:
                dfAll.append(df)
            
            #df = None
            #os.remove('NHDPlusAttributes.zip')
            #os.remove('PlusFlow.dbf')
            #os.remove('PlusFlowLineVAA.dbf')
            #os.remove('elevslope.dbf')

Retrieving region Reg14, file:NHDPlusAttributes.zip
Retrieving region Reg8, file:NHDPlusAttributes.zip
Retrieving region Reg13, file:NHDPlusAttributes.zip
Retrieving region Reg2, file:NHDPlusAttributes.zip
Retrieving region Reg18, file:NHDPlusAttributes.zip
Retrieving region Reg17, file:NHDPlusAttributes.zip
Retrieving region Reg7, file:NHDPlusAttributes.zip
Retrieving region Reg4, file:NHDPlusAttributes.zip
Retrieving region Reg9, file:NHDPlusAttributes.zip
Retrieving region Reg1, file:NHDPlusAttributes.zip
Retrieving region Reg12, file:NHDPlusAttributes.zip
Retrieving region Reg3, file:NHDPlusAttributes03N.zip
Retrieving region Reg3, file:NHDPlusAttributes03S.zip
Retrieving region Reg3, file:NHDPlusAttributes03W.zip
Retrieving region Reg5, file:NHDPlusAttributes.zip
Retrieving region Reg15, file:NHDPlusAttributes.zip
Retrieving region Reg10L, file:NHDPlusAttributes.zip
Retrieving region Reg10U, file:NHDPlusAttributes.zip
Retrieving region Reg16, file:NHDPlusAttributes.zip
Retrieving 

In [5]:
dfAll.size

3343470

In [ ]:
import tempfile
with tempfile.TemporaryDirectory() as tmpdirname:
    if 'NHDPlusAttributes' in fileName:
            #Download file
            print ('Retrieving region ' + region + ', file:' + fileName)
            fileUrl = file['url']
            ur.urlretrieve(fileUrl, fileName)


In [ ]:
df.head()

In [ ]:
                  
                    
            #zFile = ZipFile(fileName, 'r')
            #pd.read_table(zFile.open('PlusFlow.dbf'))
                
        #if 'Hydrography' in fileName:
        #   subprocess.call(r'"C:\Program Files\7-Zip\7z.exe" x ' + fileName)
        
        #Gave error BadZipFile: File is not a zip file
        #mysock = urllib.request.urlopen(fileUrl)
        #memfile = io.BytesIO(mysock.read())
        #ZipFile.infolist(memfile)
        #with ZipFile(memfile, 'r') as ftpl:
            #ftpl.namelist()
            #for file in flist:
            #    test = ftpl.extract(file)
            #    print (test)
      
        #Gave error BadZipFile: File is not a zip file
        #r = requests.get(fileUrl)
        #with closing(r), ZipFile(io.BytesIO(r.content)) as archive:
        #    print({member.filename: archive.read(member) for member in archive.infolist()})

In [ ]:
print (finalurl)

In [ ]:
for item in nhdFiles['items']:
    for tag in item['tags']:
        if 'Reg' in tag['name']:
            region = tag['name']
            print (region)

In [ ]:
print (nhdFiles)

In [ ]:
for item:
    for file in files:
        unzip file into memory
        if fileName = Flowline*
            convert correct file to dataframe
            
        
            

In [ ]:
# Set up the actions/targets for this particular instance
thisRun = {}
thisRun["instance"] = "DataDistillery"
thisRun["db"] = "BCB"
thisRun["baseURL"] = gc2.sqlAPI(thisRun["instance"],thisRun["db"])
thisRun["schema"] = "sfr"
thisRun["commitToDB"] = False

#Query comids already in the table
q_recordToSearch = "SELECT comid as lookup FROM nhd.nhdplusv2_plusflowlinevaa " 
recordToSearch = requests.get(thisRun["baseURL"]+"&q="+q_recordToSearch).json()
p = recordToSearch['features']
lookup = []
#for record returned, add record to list
for f in p:
    string = f['properties']['lookup']
    if string.startswith('"') and string.endswith('"'):
        string = string[1:-1]  #For some reason these values are coming back with double quotes, this removes outer quote
        lookup.append(string)
lenList = len(lookup)
cntExisting = 0 
existingList = []
    

In [ ]:
print (lookup)